# OpenEO: Sentinel 1, $\gamma_0$ 

Connection openeo cloud backend for cloud computing (try my early adopter programme). In this way, processing to $\gamma_0$ RTC can be conducted.

In [ ]:
import openeo
import geopandas as gpd
from pathlib import Path
import numpy as np
import pickle
import xarray as xr
import rioxarray
import hvplot.xarray
import hvplot.dask
import os
import time
#connection = openeo.connect("openeo.cloud").authenticate_oidc()
connection = openeo.connect("openeo.vito.be").authenticate_oidc()
#connection = openeo.connect("openeocloud.vito.be").authenticate_oidc()
#connection = openeo.connect("openeocloud-dev.vito.be").authenticate_oidc()
pad = Path(os.getcwd())
if pad.name == "preprocessing_files":
    pad_correct = pad.parent
    os.chdir(pad_correct)
#set all of the parameters below to True to execute cloud computing and downloading    
overwrite = True
read = True
job_exec =  True
download = True
write_out = True

Load in the Shapefile of the Zwalm in EPSG:4326

In [ ]:
if not os.path.exists('data/Zwalm_shape/zwalm_shapefile_emma.shp'):
    %run "preprocessing_files/shapefile_conversion.py"   
    print('Preprocessing script of Zwalm shapefile has run')

In [ ]:
shape_zwalm = gpd.read_file('data/Zwalm_shape/zwalm_shapefile_emma.shp')
shape_zwalm.plot()
extent = shape_zwalm.total_bounds
print(extent)
#[ 3.66751526 50.76325563  3.83821038 50.90341411] (give this extent of shapefile not on disk)

In [ ]:
#connection.list_collections()

In [ ]:
connection.describe_collection('SENTINEL1_GRD')
#connection.describe_collection('S1_GRD_SIGMA0_ASCENDING')

Splitting the temporal extent in to 1 year at a time

In [ ]:
temporal_extent = ["2015-06-07", "2022-11-05"]
list_temp_extent = []
job_title_list = []
job_title = "s1_a_gamma0_2015" 
job_title_list.append(job_title)
list_temp_extent.append([temporal_extent[0],"2015-12-31"])
years = np.arange(2016,2023)
for year in np.arange(2016,2023):
    if year == 2022:
        #print([str(year)+"-01-01",temporal_extent[1]])
        list_temp_extent.append([str(year)+"-01-01",str(year)+ "-06-30"])
        job_title = "s1_a_gamma0_2022_I"
        job_title_list.append(job_title)
        list_temp_extent.append([str(year)+"-07-01",temporal_extent[1]])
        job_title = "s1_a_gamma0_2022_II" 
        job_title_list.append(job_title)
    else:
        #print([str(year)+"-01-01",str(year)+ "-12-31"])
        list_temp_extent.append([str(year)+"-01-01",str(year)+ "-06-30"])
        job_title = "s1_a_gamma0_" +  str(year) + "_I"
        job_title_list.append(job_title)
        list_temp_extent.append([str(year)+"-07-01",str(year) + "-12-31"])
        job_title = "s1_a_gamma0_" +  str(year) + "_II"
        job_title_list.append(job_title)
print(list_temp_extent)
print(job_title_list)

In [ ]:
print(len(list_temp_extent))
print(len(job_title_list))
print(list_temp_extent[2:-1])
#display(connection.list_jobs())

## Ascending orbit(s)

 for filtering: https://docs.sentinel-hub.com/api/latest/data/sentinel-1-grd/#filter-extension

 https://docs.openeo.cloud/data-collections/ check here for correct filtering e.g. sar:...  or sat:... 


In [ ]:
collection = 'SENTINEL1_GRD' #Ground Range Detected #Ground Range Detected
spatial_extent = {'west':extent[0],'east':extent[2],'south':extent[1],'north':extent[3]}
bands = ["VV","VH"]#enkel in deze geïnteresseerd 
properties = {
    "sat:orbit_state": lambda od: od == "ASCENDING", ##filter on ascending vs descending
    "sar:instrument_mode":lambda mode: mode == "IW", ## Orbit direction filtering
    "polarization": lambda p: p == "DV"
    #"sar:polarizations": lambda p: p == "DV", ## Suggestion Jeroen 27/02/2023
    #"s1:polarization": lambda p: p == "DV", ## Suggestion Jeroen 27/02/2023
    #"s1:resolution": lambda res : res == "HIGH" ## 10 m resolution for IW
 }

https://docs.openeo.cloud/processes/#sar_backscatter 

Sentinel-1 GRD provided by Sentinel Hub: https://docs.sentinel-hub.com/api/latest/data/sentinel-1-grd/ 

In [ ]:
job_id_list = []
if job_exec:
    for i, temporal_extent in enumerate(list_temp_extent):
        #if i >= 2: #temporary since first 2 already worked! 
        s1a = connection.load_collection(
            collection_id = collection,
            spatial_extent= spatial_extent,
            temporal_extent = temporal_extent,
            bands = bands,
            properties= properties
        )
        #s1a = s1a.ard_normalized_radar_backscatter(elevation_model = "COPERNICUS_30")
        s1a = s1a.sar_backscatter(
            coefficient  = "gamma0-terrain", #default
            local_incidence_angle  = True,
            elevation_model = "COPERNICUS_30"
        ) #suggestion Jeroen
        s1a = s1a.mask_polygon(shape_zwalm['geometry'].values[0])
        # job_title = "s1_a_gamm0" +  str(years[i])
        # job_title_list.append(job_title)
        job_s1a = s1a.create_job(title = job_title_list[i], out_format= 'NetCDF')
        job_s1a_id = job_s1a.job_id
        if job_s1a_id:
            print("Batch job created with id: ",job_s1a_id)
            #job_s1a.start_and_wait()
            job_s1a.start_job()
            job_id_list.append(job_s1a_id)
            time.sleep(40) # to prevent overloading the SentinelHub server
        else:
            print("Error! Job ID is None")

In [ ]:
if not os.path.exists('data/g0_OpenEO/ascending'):
    os.makedirs('data/g0_OpenEO/ascending')

In [ ]:
if overwrite:
    with open('data/g0_OpenEO/s1_a_job_id_list.pickle', 'wb') as handle:
        pickle.dump(job_id_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('data/g0_OpenEO/s1_a_job_title_list.pickle', 'wb') as handle:
        pickle.dump(job_title_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
if read:
    job_id_list = pickle.load(open('data/g0_OpenEO/s1_a_job_id_list.pickle', "rb"))
    job_title_list = pickle.load(open('data/g0_OpenEO/s1_a_job_title_list.pickle', "rb"))

In [ ]:
if download:
    for i,job_id in enumerate(job_id_list):
        job_connection = connection.job(job_id)
        results = job_connection.get_results()
        name_netcdf = job_title_list[i] + '.nc'
        filepath = "data/g0_OpenEO/ascending/" + name_netcdf
        print(filepath)
        while job_connection.status() != 'finished':
            time.sleep(30)
            if job_connection.status() == 'error':
                raise ChildProcessError(job_id + 'has encountered an error, check why batch job failed')
            if job_connection.status() == 'canceled':
                        raise ChildProcessError(job_id + 'has been canceled')
        results.download_file(filepath)

<!-- Read in the downloaded data! 
- scale and offset from https://docs.terrascope.be/DataProducts/Sentinel-1/references/VITO_S1_sigma0_GRD.pdf  -->

In [ ]:
s1_xr_asc = xr.open_mfdataset('data/g0_OpenEO/ascending/*.nc', decode_coords="all") #automatically chuncked!
s1_xr_asc_plot = s1_xr_asc.copy()
s1_xr_asc_plot['VV_db'] = 10 * np.log10(s1_xr_asc['VV'])
# scale = 0.0005
# offset = 29
# s1_xr_asc['angle']  = s1_xr_asc['angle']*scale + offset   
s1_xr_asc

Now only select the values for which sufficient data is present. First check amount of data in a full image

In [ ]:
s1_xr_asc_plot['VV_db'].isel(t=0).plot() #example of a full image

In [ ]:
s1_xr_asc['local_incidence_angle'].isel(t=0).plot()

VV

In [ ]:
xr_shape = s1_xr_asc['VV'].shape
print('Shape of the ascending orbits: ' + str(xr_shape))
nr_pixels = xr_shape[1]*xr_shape[2]
print('Number of pixels per timestamp: ' + str(nr_pixels))
nancount = np.sum(np.isnan(s1_xr_asc['VV'].isel(t=0))).values
print('Number of nan-pixels for a full image: ' + str(nancount))
nan_cutoff = nancount/nr_pixels
print('Percentage of nan-pixels in a full image: ' + str(nan_cutoff*100) + '%')
#add 5% as safety margin to cutoff
nan_cutoff = nan_cutoff + 0.05

VH

In [ ]:
xr_shape = s1_xr_asc['VV'].shape
print('Shape of the ascending orbits: ' + str(xr_shape))
nr_pixels = xr_shape[1]*xr_shape[2]
print('Number of pixels per timestamp: ' + str(nr_pixels))
nancount = np.sum(np.isnan(s1_xr_asc['VV'].isel(t=0))).values
print('Number of nan-pixels for a full image: ' + str(nancount))
nan_cutoff = nancount/nr_pixels
print('Percentage of nan-pixels in a full image: ' + str(nan_cutoff*100) + '%')
#add 5% as safety margin to cutoff
nan_cutoff = nan_cutoff + 0.05

In [ ]:
# VV
nr_timestemps_ascending = xr_shape[0]
bool_full_image = []
for i in range(nr_timestemps_ascending):
    VV_ds = s1_xr_asc['VV'].isel(t=i)
    temp_nancount = np.sum(np.isnan(VV_ds)).values
    nan_frac = temp_nancount/nr_pixels
    if nan_frac > nan_cutoff:
        bool_full_image.append(0)
    else:
        bool_full_image.append(1)

#VH
bool_full_image_VH = []
for i in range(nr_timestemps_ascending):
    VV_ds = s1_xr_asc['VH'].isel(t=i)
    temp_nancount = np.sum(np.isnan(VV_ds)).values
    nan_frac = temp_nancount/nr_pixels
    if nan_frac > nan_cutoff:
        bool_full_image_VH.append(0)
    else:
        bool_full_image_VH.append(1)

#Compare
if not bool_full_image_VH == bool_full_image:
    pos_emp_VH = np.where(np.array(bool_full_image_VH) == 0)[0].tolist()
    bool_full_image_np = np.array(bool_full_image)
    bool_full_image_np[pos_emp_VH] = 0
    bool_full_image_all = bool_full_image_np.tolist()
    print('VH and VV Nans were not equal')

In [ ]:
pos_full = np.where(bool_full_image)[0].tolist()
print(pos_full)

In [ ]:
#add orbit direction
da = xr.DataArray(
    data = np.repeat('ascending',nr_timestemps_ascending),
    dims = ['t'],
    coords = dict(t = s1_xr_asc['t'].values)
)
s1_xr_asc['Orbitdirection'] = da

In [ ]:
s1_xr_asc_full = s1_xr_asc.isel(t = pos_full)
s1_xr_asc_full

In [ ]:
s1_xr_asc_full_plot = s1_xr_asc_full.copy()
s1_xr_asc_full_plot['VV_db'] = 10 * np.log10(s1_xr_asc_full['VV'])
s1_xr_asc_full_plot['VV_db'].hvplot.image('x','y', geo = True, crs = 32631, tiles = 'OSM', cmap = 'bwr', width = 400, rasterize = True)

## Descending orbit(s)

In [ ]:
properties = {
    "sat:orbit_state": lambda od: od == "DESCENDING", ##filter on ascending vs descending
    "sar:instrument_mode":lambda mode: mode == "IW", ## Orbit direction filtering
    "polarization": lambda p: p == "DV"
    #"sar:polarizations": lambda p: p == "DV", ## Suggestion Jeroen 27/02/2023
    #"s1:polarization": lambda p: p == "DV", ## Suggestion Jeroen 27/02/2023
    #"s1:resolution": lambda res : res == "HIGH" ## 10 m resolution for IW
 }

In [ ]:
years = np.arange(2015,2023)
years_name_list = []
for i, year in enumerate(years):
    if i > 0:
        years_name_list.append(str(year)+ '_I')
        years_name_list.append(str(year) + '_II')
    else:
        years_name_list.append(str(year))
print(years_name_list)
print(len(years_name_list) == len(list_temp_extent))

In [ ]:
job_title_list_d = []
job_id_list_d = []
if job_exec:
    for i, temporal_extent in enumerate(list_temp_extent):
        s1d = connection.load_collection(
            collection_id = collection,
            spatial_extent= spatial_extent,
            temporal_extent = temporal_extent,
            bands = bands,
            properties= properties
        )
        #s1a = s1a.ard_normalized_radar_backscatter(elevation_model = "COPERNICUS_30")
        s1d = s1d.sar_backscatter(
            coefficient  = "gamma0-terrain", #default
            local_incidence_angle  = True,
            elevation_model = "COPERNICUS_30"
        ) #suggestion Jeroen
        s1d = s1d.mask_polygon(shape_zwalm['geometry'].values[0])
        job_title = "s1_d_gamma0-" +  years_name_list[i]
        job_title_list_d.append(job_title)
        job_s1d = s1d.create_job(title = job_title, out_format= 'NetCDF')
        job_s1d_id = job_s1d.job_id
        if job_s1d_id:
            print("Batch job created with id: ",job_s1a_id)
            #job_s1a.start_and_wait()
            job_s1d.start_job()
            job_id_list_d.append(job_s1d_id)
            time.sleep(40) # to prevent overloading the SentinelHub server
        else:
            print("Error! Job ID is None")

In [ ]:
## TEMP
# len(list_temp_extent)
# len(job_id_list)
# job_id_list_d = job_id_list[-15:]
# job_id_list_d

In [ ]:
if overwrite:
    with open('data/g0_OpenEO/s1_d_job_id_list.pickle', 'wb') as handle:
        pickle.dump(job_id_list_d, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('data/g0_OpenEO/s1_d_job_title_list.pickle', 'wb') as handle:
        pickle.dump(job_title_list_d, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
if read:
    job_id_list_d = pickle.load(open('data/g0_OpenEO/s1_d_job_id_list.pickle', "rb"))
    job_title_list_d = pickle.load(open('data/g0_OpenEO/s1_d_job_title_list.pickle', "rb"))

In [ ]:
if not os.path.exists('data/g0_OpenEO/descending'):
    os.makedirs('data/g0_OpenEO/descending')
if download:
    for i,job_id in enumerate(job_id_list_d):
        job_connection = connection.job(job_id)
        results = job_connection.get_results()
        name_netcdf = job_title_list_d[i] + '.nc'
        filepath = "data/g0_OpenEO/descending/" + name_netcdf
        print(filepath)
        while job_connection.status() != 'finished':
            time.sleep(30)
            if job_connection.status() == 'error':
                raise ChildProcessError(job_id + 'has encountered an error, check why batch job failed')
            if job_connection.status() == 'canceled':
                        raise ChildProcessError(job_id + 'has been canceled')
        results.download_file(filepath)

In [ ]:
s1_xr_desc = xr.open_mfdataset('data/g0_OpenEO/descending/*.nc', decode_coords="all") #automatically chuncked!
# s1_xr_desc['VV_db'] = 10 * np.log10(s1_xr_desc['VV'])
#s1_xr_desc['angle']  = s1_xr_desc['angle']*scale + offset   
display(s1_xr_desc)
xr_shape_desc = s1_xr_desc['VV'].shape
#only full images selected
nr_timestemps_descending = xr_shape_desc[0]
#VV
nr_pixels = xr_shape[1]*xr_shape[2]
print('Number of pixels per timestamp: ' + str(nr_pixels))
nancount = np.sum(np.isnan(s1_xr_desc['VV'].isel(t=0))).values
print('Number of nan-pixels for a full image: ' + str(nancount))
nan_cutoff = nancount/nr_pixels
print('Percentage of nan-pixels in a full image: ' + str(nan_cutoff*100) + '%')
#add 5% as safety margin to cutoff
nan_cutoff = nan_cutoff + 0.05

In [ ]:
bool_full_image = []
for i in range(nr_timestemps_descending):
    VV_ds = s1_xr_desc['VV'].isel(t=i)
    temp_nancount = np.sum(np.isnan(VV_ds)).values
    nan_frac = temp_nancount/nr_pixels
    if nan_frac > nan_cutoff:
        bool_full_image.append(0)
    else:
        bool_full_image.append(1)
#VH
bool_full_image_VH = []
for i in range(nr_timestemps_descending):
    VV_ds = s1_xr_desc['VH'].isel(t=i)
    temp_nancount = np.sum(np.isnan(VV_ds)).values
    nan_frac = temp_nancount/nr_pixels
    if nan_frac > nan_cutoff:
        bool_full_image_VH.append(0)
    else:
        bool_full_image_VH.append(1)
if not bool_full_image_VH == bool_full_image:
    pos_emp_VH = np.where(np.array(bool_full_image_VH) == 0)[0].tolist()
    bool_full_image_np = np.array(bool_full_image)
    bool_full_image_np[pos_emp_VH] = 0
    bool_full_image_all = bool_full_image_np.tolist()
    print('VH and VV Nans were not equal')
pos_full = np.where(bool_full_image_all)[0].tolist()
#add orbit direction
da = xr.DataArray(
    data = np.repeat('descending',nr_timestemps_descending),
    dims = ['t'],
    coords = dict(t = s1_xr_desc['t'].values)
)
s1_xr_desc['Orbitdirection'] = da
s1_xr_desc_full = s1_xr_desc.isel(t= pos_full)
display(s1_xr_desc_full)

## Combining orbits

In [ ]:
s1_xr_full = xr.merge([s1_xr_asc_full, s1_xr_desc_full])
display(s1_xr_full)

In [ ]:
if write_out:
    s1_xr_full.to_netcdf('data/g0_OpenEO/g0_zwalm.nc')